In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:

# Define the directory containing your dataset
dataset_dir = 'resized_data'

# Define the ImageDataGenerator with augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)

# Loop through each subdirectory in the dataset directory
for root, dirs, files in os.walk(dataset_dir):
    for dir_name in dirs:
        # Create a data generator for the current subdirectory
        subdir_path = os.path.join(root, dir_name)
        print("Augmenting images in:", subdir_path)

        # Generate augmented images and save them to the same directory
        augment_generator = datagen.flow_from_directory(
            directory=dataset_dir,  # Use the root directory of the dataset
            classes=[dir_name],  # Specify the current subdirectory as the class
            target_size=(250, 250),  # Resize images to 250x250
            batch_size=32,
            class_mode=None,  # No labels needed, as we're not training
            save_to_dir=subdir_path,  # Save augmented images to the same directory
            save_prefix='aug',  # Prefix for augmented image filenames
            save_format='jpg'  # Save images as JPEG files
        )

        # Generate and save augmented images
        num_images = len(os.listdir(subdir_path))
        num_batches = (1000 - num_images) // 32  # Assuming target of 1000 images per class and batch size of 32
        for i in range(num_batches):
            batch = augment_generator.next()  # Generate a batch of augmented images


In [ ]:
import tensorflow as tf
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define constants
IMAGE_SIZE = (250, 250)
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 8  # Change this to the number of students you have

In [ ]:
# Create data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/resized_data',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    '/content/resized_data',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Define model
model = Sequential()

# Add pre-trained VGG16 base
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(250, 250, 3))
for layer in base_model.layers:
    layer.trainable = False
model.add(base_model)

# Add additional layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))  # Assuming 8 output classes


In [ ]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Print the model summary
model.summary()

In [ ]:
# Load augmented training data
train_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values
train_generator = train_datagen.flow_from_directory(
    '/content/resized_data',
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical'
)

# Train model
history = model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
)

In [ ]:
# Assuming you have a separate test generator for your test dataset
test_generator = train_datagen.flow_from_directory(
    '/content/resized_data',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Ensure that the data is not shuffled for evaluation
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import numpy as np
from keras.preprocessing import image

# Load and preprocess the test image
img_path = '/content/resized_data/1RN21IS167/0000.jpg'  # Replace with the path to your test image
img = image.load_img(img_path, target_size=(250, 250))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize pixel values to [0, 1]

# Predict the class probabilities
predictions = model.predict(img_array)

# Map the predicted class probabilities to class labels
# Assuming each class corresponds to a different student
class_index = np.argmax(predictions)
# Assuming you have a list of student names corresponding to class indices
student_names = ['Rachel_Green', 'Monica_Geller', 'Phoebe_Buffay', 'Joey_Tribbiani', 'Chandler_Bing', 'Ross_Geller', 'Janice_Litman', 'Gunther']
predicted_student = student_names[class_index]

print("Predicted student:", predicted_student)


In [ ]:
model.save('/content/drive/My Drive/face_recognition.h5')
